In [27]:
from __future__ import print_function
import torch
import torchvision

from torch import nn
from torch.nn import functional as F
from torchvision import datasets, transforms

class LeNet_2(nn.Module):
  
  def __init__(self):
    super().__init__()
    # 32px 이었다가, conv를 거치면서 절반으로 감소
    self.conv1 = nn.Conv2d(3, 16, 3, 1, padding=1)
    self.conv2 = nn.Conv2d(16, 32, 3, 1, padding=1)
    self.conv3 = nn.Conv2d(32, 64, 3, 1, padding=1)
    self.fc1 = nn.Linear(4*4*64, 10)
    self.dropout1 = nn.Dropout(0.5)
    
  def forward(self, x):
    x = F.relu(self.conv1(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv2(x))
    x = F.max_pool2d(x, 2, 2)
    x = F.relu(self.conv3(x))
    x = F.max_pool2d(x, 2, 2)
    x = self.dropout1(x)
    x = x.view(-1, 4*4*64)
    x = F.relu(self.fc1(x))
    return x

device = torch.device("cuda:0")
model = LeNet_2().to(device)

transformer = transforms.Compose([transforms.Resize((32, 32)),
                                  transforms.ToTensor(),
                                  transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                                 ])

train_set = datasets.CIFAR10(root='./data', train=True, download=True, transform=transformer)
test_set = datasets.CIFAR10(root='./data', train=False, download=True, transform=transformer)

train_loader = torch.utils.data.DataLoader(dataset=train_set, batch_size=100, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_set, batch_size=1, shuffle=False)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adadelta(model.parameters(), lr=1.0)

epochs = 12
running_loss_history = []
running_correct_history = []

for e in range(epochs):
  
  running_loss = 0.0
  running_correct = 0.0
  validation_running_loss = 0.0
  validation_running_correct = 0.0
  
  for inputs, labels in train_loader:    
     
    inputs = inputs.to(device)
    labels = labels.to(device)
    outputs = model(inputs)
    loss = criterion(outputs, labels)
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    _, preds = torch.max(outputs, 1)
    
    running_correct += torch.sum(preds == labels.data)
    running_loss += loss.item()
    
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = running_correct.float() / len(train_loader)
    running_loss_history.append(epoch_loss)
    running_correct_history.append(epoch_acc)

  print("===================================================")
  print("epoch: ", e + 1)
  print("training loss: {:.5f}, acc: {:5f}".format(epoch_loss, epoch_acc))
with torch.no_grad():
  acc_1=0
  acc_5=0
  for val_input, val_label in test_loader:
    
    val_input = val_input.to(device)
    val_label = val_label.to(device)
    val_outputs = model(val_input)

    _, val_preds_1 = torch.max(val_outputs, 1)
    _, five = torch.sort(val_outputs)
    five = five[-5:]
    test_correct_1 = torch.sum(val_preds_1 == val_label.data)
    test_correct_5 = torch.sum(val_label.data in five == True)

    acc_1+=test_correct_1.float()
    acc_5+=test_correct_5.float()
  
  print("TOP 1 acc: {:5f} %".format(acc_1*100/len(test_loader)))
  print("TOP 5 acc: {:5f} %".format(acc_5*100/len(test_loader)))

Files already downloaded and verified
Files already downloaded and verified
10000
epoch:  1
training loss: 1.84748, acc: 35.382000
epoch:  2
training loss: 1.50286, acc: 48.660004
epoch:  3
training loss: 1.22204, acc: 57.804001
epoch:  4
training loss: 1.08712, acc: 62.210003
epoch:  5
training loss: 1.02402, acc: 64.528000
epoch:  6
training loss: 0.97783, acc: 66.130005
epoch:  7
training loss: 0.94425, acc: 67.724007
epoch:  8
training loss: 0.91550, acc: 68.396004
epoch:  9
training loss: 0.89954, acc: 69.016006
epoch:  10
training loss: 0.87812, acc: 69.750000
epoch:  11
training loss: 0.86347, acc: 70.246002
epoch:  12
training loss: 0.85405, acc: 70.816002
TOP 1 acc: 68.339996 %
TOP 5 acc: 100.000000 %
